In [25]:

import numpy as np
import pandas as pd

import json
import grequests

In [4]:
path = r'G:\Github\Investigating_Singapore_Housing_Price\data\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv'
df = pd.read_csv(path)

In [13]:
df.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
62260,2019-11,YISHUN,5 ROOM,865,YISHUN ST 81,07 TO 09,122.0,Improved,1988,67 years 05 months,458000.0
62261,2019-11,YISHUN,EXECUTIVE,328,YISHUN RING RD,07 TO 09,142.0,Apartment,1988,67 years 08 months,565000.0
62262,2019-11,YISHUN,EXECUTIVE,328,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,67 years 07 months,538000.0
62263,2019-11,YISHUN,EXECUTIVE,611,YISHUN ST 61,10 TO 12,146.0,Maisonette,1987,67 years,588888.0
62264,2019-11,YISHUN,EXECUTIVE,828,YISHUN ST 81,04 TO 06,145.0,Apartment,1988,67 years 02 months,618000.0


In [3]:
df.corr()

,floor_area_sqm,lease_commence_date,resale_price
floor_area_sqm,1.000000,0.218871,0.632800
lease_commence_date,0.218871,1.000000,0.325639
resale_price,0.632800,0.325639,1.000000


In [100]:
df.head()
df.street_name = df.street_name.replace({"ST. ":"ST "},regex = True)

In [101]:
short_url = "https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y"
address_to_search = [blk+"+"+street.replace(' ',"+") for blk,street in zip(df.block,df.street_name)]
urls = [short_url.format(add) for add in address_to_search]

In [11]:
with open('url.txt','w') as f:
    f.writelines([u + "\n" for u in urls])

In [12]:
len(urls)

62265

In [36]:
houseinfo = []
url_used = []
with open('houseinfo.txt','r') as f:
    for line in f:
        houseinfo.append(json.loads(line.split('"text":')[1][:-2]))
        url_used.append(line.split('"text":')[0][1:])

In [25]:
houseinfo[0]['results']

[{'SEARCHVAL': 'HDB-CHOA CHU KANG',
  'BLK_NO': '689A',
  'ROAD_NAME': 'CHOA CHU KANG DRIVE',
  'BUILDING': 'HDB-CHOA CHU KANG',
  'ADDRESS': '689A CHOA CHU KANG DRIVE HDB-CHOA CHU KANG SINGAPORE 681689',
  'POSTAL': '681689',
  'X': '18763.3961538309',
  'Y': '42761.076653048396',
  'LATITUDE': '1.40298901385213',
  'LONGITUDE': '103.750320068403',
  'LONGTITUDE': '103.750320068403'}]

In [29]:
postal_list = []
failure = []
for i,item in enumerate(houseinfo):
    try:
        postal_list.append(item['results'][0]['POSTAL'])
    except IndexError:
        failure.append(i)

In [30]:
len(postal_list)

61970

In [31]:
len(failure)

295

In [32]:
df.iloc[failure]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
289,2017-01,BUKIT PANJANG,5 ROOM,174,LOMPANG RD,04 TO 06,122.0,Improved,1997,79 years 07 months,555000.0
408,2017-01,GEYLANG,4 ROOM,120,GEYLANG EAST CTRL,10 TO 12,91.0,New Generation,1982,64 years 04 months,515000.0
899,2017-01,SERANGOON,4 ROOM,109,SERANGOON NTH AVE 1,04 TO 06,91.0,New Generation,1985,67 years 05 months,390000.0
925,2017-01,TAMPINES,3 ROOM,806,TAMPINES AVE 4,04 TO 06,73.0,Model A,1984,66 years 02 months,345000.0
941,2017-01,TAMPINES,4 ROOM,421,TAMPINES ST 41,04 TO 06,105.0,Model A,1985,67 years 05 months,413888.0
...,...,...,...,...,...,...,...,...,...,...,...
60499,2019-11,BEDOK,4 ROOM,667,JLN DAMAI,01 TO 03,108.0,Model A,1996,76 years,448000.0
60511,2019-11,BEDOK,5 ROOM,147,BEDOK RESERVOIR RD,07 TO 09,122.0,Improved,1987,66 years 04 months,468000.0
60521,2019-11,BEDOK,5 ROOM,31,BEDOK ST AVE 2,19 TO 21,118.0,Standard,1978,57 years 04 months,600000.0
60522,2019-11,BEDOK,5 ROOM,32,BEDOK ST AVE 2,10 TO 12,115.0,Standard,1978,57 years 04 months,497000.0


In [43]:
url_u = [u.strip()[1:-1] for u in url_used]

In [51]:
len(url_u)

62265

In [52]:
url_u[0]

'url": https://developers.onemap.sg/commonapi/search?searchVal=689A+CHOA+CHU+KANG+DR&returnGeom=Y&getAddrDetails=Y'

In [54]:
urls[0]

'https://developers.onemap.sg/commonapi/search?searchVal=406+ANG+MO+KIO+AVE+10&returnGeom=Y&getAddrDetails=Y'